# Lectura 54: SQL en Polars I

## ¿Cómo utilizar SQL en Polars?

Polars utiliza el objeto `SQLContext` para gestionar consultas SQL

In [1]:
import polars as pl

ctx = pl.SQLContext()

ctx.tables()

[]

## ¿Cómo registrar un DataFrame?

Hay varias formas de registrar DataFrames durante la inicialización del `SQLContext`.

- Registrar todos los objetos `LazyFrame` y `DataFrame` en el namespace global.
- Registrar explícitamente a través de un mapeo en un diccionario, o como argumentos del tipo llave-valor.

In [2]:
df = pl.DataFrame(
    {
        'id': [1, 2, 3],
        'color': ['rojo', 'verde', 'azul']
    }
)

lf = pl.LazyFrame(
    {
        'letra': ['a', 'b', 'c'],
        'conteo': [6, 9, 3]
    }
)

## Registrar todos los dataframes en el namespace global

Esta opción registrará tanto a los `DataFrame` como a los `LazyFrame`.

In [3]:
ctx = pl.SQLContext(register_globals=True)

ctx.tables()

['df', 'lf']

## Registrar dataframes explícitamente

### A través de un diccionario

In [4]:
ctx = pl.SQLContext(frames={'df_color': df, 'df_letas': lf})

ctx.tables()

['df_color', 'df_letas']

### Como argumentos llave-valor

In [5]:
ctx = pl.SQLContext(df=df, lf=lf)

ctx.tables()

['df', 'lf']

Una vez que se inicializa el `SQLContext`, podemos registrar DataFrames adicionales o eliminarlos del registro existente. Para ello emplearemos las siguientes funciones:

- `register`
- `register_many`
- `unregister`
- `register_globals`

In [6]:
df1 = pl.DataFrame({'a': [1]})
df2 = pl.DataFrame({'b': [2]})
df3 = pl.DataFrame({'c': [3]})

In [7]:
ctx.register('df1', df1)

ctx.tables()

['df', 'df1', 'lf']

In [8]:
ctx.register_many({'df2': df2, 'df3': df3})

ctx.tables()

['df', 'df1', 'df2', 'df3', 'lf']

In [9]:
ctx.unregister(['df1', 'df2', 'df3'])

ctx.tables()

['df', 'lf']

In [10]:
ctx.register_globals()

ctx.tables()

['df', 'df1', 'df2', 'df3', 'lf']

## Ejecución de consultas

Las consultas SQL siempre se ejecutan en modo lazy para aprovechar todas las optimizaciones que nos ofrece Polars, por lo que tenemos dos opciones para obtener el resultado:

- Establecer el parámetro `eager_execution=True` en el `SQLContext` lo que garantizará que Polars recopile automáticamente los resultados del LazyFrame cada vez que llamemos a la función `execute`.
- Establecer el parámetro `eager=True` al ejecutar una consulta con `execute` o llamar explícitamente a la funciòn `collect`.

Las consultas SQL se ejecutan llamando a `execute` en un `SQLContext`.

In [11]:
ctx = pl.SQLContext(register_globals=True, eager_execution=True)

ctx.execute("SELECT * FROM df")

id,color
i64,str
1,"""rojo"""
2,"""verde"""
3,"""azul"""


In [12]:
ctx.execute("SELECT * FROM lf WHERE conteo >= 6")

letra,conteo
str,i64
"""a""",6
"""b""",9


In [13]:
ctx = pl.SQLContext(register_globals=True)

In [14]:
ctx.execute("SELECT * FROM df")

In [15]:
ctx.execute("SELECT * FROM df", eager=True)

id,color
i64,str
1,"""rojo"""
2,"""verde"""
3,"""azul"""


In [16]:
ctx.execute("SELECT * FROM lf WHERE conteo >= 6").collect()

letra,conteo
str,i64
"""a""",6
"""b""",9
